In [1]:
import torch
import torch.nn as nn
import torch.utils.data as Data
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import random
import pandas as pd
import os
import math

class MyModel_0(nn.Module):
    def __init__(self):#在这里定义网络层
        super(MyModel_0, self).__init__()
        self.liner1=nn.Linear(1504,1,bias=False)
        nn.init.zeros_(self.liner1.weight)

    def forward(self,x):#前向传播
        output=self.liner1(x)
        return output
    
class MyModel_1(nn.Module):
    def __init__(self):#在这里定义网络层
        super(MyModel_1, self).__init__()
        self.liner1=nn.Linear(1504,1,bias=False)
        nn.init.zeros_(self.liner1.weight)

    def forward(self,x):#前向传播
        output=self.liner1(x)
        return output

class CustomLoss(torch.nn.Module):
    def __init__(self):
        super(CustomLoss, self).__init__()

    def forward(self, output, target):
        loss = torch.mean(0.5*(output - target)**2)   # 定义自己的损失函数，这里是MSE
        return loss
model_0=MyModel_0()#建立模型
model_1=MyModel_1()#建立模型
loss_fn=CustomLoss()#构建损失函数
optimizer_0=optim.Adam(model_0.parameters(),lr=0.001)#构建优化器(模型参数,学习率)
optimizer_1=optim.Adam(model_1.parameters(),lr=0.001)#构建优化器(模型参数,学习率)
B=5000
Epoch=5000
danwei_b=B/(1241*30)
experience=[]
nt=[0,0,0,0]
rootpath='C:\\Users\\98306\\Desktop\\实验\\训练数据\\500'
loss_abc=[]
for i in range(Epoch):
    print('epoch:',i)
    idx=random.randrange(0,100)
    roadpath=str(idx)+'_road.xlsx'
    roadpath=os.path.join(rootpath,roadpath)
    road=pd.read_excel(roadpath)
    road=road.to_numpy()
    
    bpath=str(idx)+'_b.xlsx'
    bpath=os.path.join(rootpath,bpath)
    b=pd.read_excel(bpath,header=None)
    b=b.to_numpy().flatten()
    
    path=r'C:\Users\98306\Desktop\实验\Real map\poi次数分布(总次数为1241).xlsx'
    df=pd.read_excel(path)
    arr_df = df.to_numpy().flatten()

    B_i=B
    res=0#记录每轮最大的TD-error
    total_value=0#每轮获得的总价值
    
    for j in range(road.shape[0]):
        for k in range(road.shape[1]):
            if arr_df[k]==0:
                road[j,k]=0
        if road[j].sum()*danwei_b<b[j]:
            continue
        pi=0
        reward=0
        s= np.array([]).astype(np.double)
        s= np.append(s,1)
        s= np.append(s,j)
        s= np.concatenate((s, road[j]))
        s= np.append(s,b[j])
        s= np.concatenate((s,arr_df))
        s= np.append(s,B_i)
        epsilon=0.1
        lamda=random.uniform(0, 1)
        if lamda<epsilon/2:#随机不选
            pi=0
            reward=0
            nt[0]+=1
        elif lamda<epsilon:#随机选择
            pi=1
            reward=road[j].sum()
            nt[1]+=1
        else:
            with torch.no_grad():
                t0=model_0(torch.Tensor(s))
                t1=model_1(torch.Tensor(s))
                t0=t0.detach().numpy()
                t1=t1.detach().numpy()
                
                if t0[0]<t1[0]:
                    pi=1
                    reward=road[j].sum()
                    nt[2]+=1
                else:
                    pi=0
                    reward=0
                    nt[3]+=1

        if j>=1:#存入经验回放数组
            exper=[]
            exper.append(s_old)
            exper.append(pi_old)
            exper.append(reward_old)
            exper.append(s)
            exper.append(pi)
            experience.append(exper)
#         if j==road.shape[0]-1:
#             exper=[]
#             exper.append(s)
#             exper.append(pi)
#             exper.append(reward)
#             exper.append(np.array([0]))
#             exper.append(0)
#             experience.append(exper)
            
        s_old=s
        pi_old=pi
        reward_old=reward
        
        if pi==1:
            for k in range(road.shape[1]):
                if road[j,k]>0:
                    arr_df[k]-=1
            B_i-=road[j].sum()*danwei_b
            

        
        total_value+=reward
        if len(experience)>=1000:
            experience.pop(0)
        if len(experience)>100:
            idx=random.randrange(0,len(experience))#随机抽取一条经验
            replay_s_old=experience[idx][0]#S
            replay_pi_old=experience[idx][1]#A
            replay_reward=experience[idx][2]#R
            replay_s=experience[idx][3]#S
            replay_pi=experience[idx][4]#A
#             if replay_s.shape[0]==1:
#                 x_train=torch.Tensor(replay_s_old)
#                 real=torch.Tensor(np.array([replay_reward]))
#                 if replay_pi_old==0:
#                     pred=model_0(x_train)
#                     loss=loss_fn(pred,real)
                    
#                     optimizer_0.zero_grad()
#                     loss.backward()
#                     optimizer_0.step()
#                     res=max(res,loss.item())
#                 else:
#                     pred=model_1(x_train)
#                     loss=loss_fn(pred,real)
                    
#                     optimizer_1.zero_grad()
#                     loss.backward()
#                     optimizer_1.step()
#                     res=max(res,loss.item())
#             else:
            if replay_s.shape[0]!=1:
                x_train=torch.Tensor(replay_s_old)
                y_train=torch.Tensor(replay_s)
                if replay_pi_old==0 and replay_pi==0:
#                     print("111")
                    with torch.no_grad():
                        real=torch.Tensor(np.array([replay_reward]))+model_0(y_train)
                    pred=model_0(x_train)
                    loss=loss_fn(pred,real)
                    
                    optimizer_0.zero_grad()
                    loss.backward()
                    optimizer_0.step()
                    res=max(res,loss.item())
                elif replay_pi_old==0 and replay_pi==1:
#                     print("222")
                    with torch.no_grad():
                        real=torch.Tensor(np.array([replay_reward]))+model_1(y_train)
                    pred=model_0(x_train)
                    loss=loss_fn(pred,real)
                    
                    optimizer_0.zero_grad()
                    loss.backward()
                    optimizer_0.step()
                    res=max(res,loss.item())
                elif replay_pi_old==1 and replay_pi==0:
#                     print("333")
                    with torch.no_grad():
                        real=torch.Tensor(np.array([replay_reward]))+model_0(y_train)
#                         print(real,torch.Tensor(np.array([replay_reward])),model_0(y_train))
                    pred=model_1(x_train)
                    loss=loss_fn(pred,real)
                    
                    optimizer_1.zero_grad()
                    loss.backward()
                    optimizer_1.step()
                    res=max(res,loss.item())
                elif replay_pi_old==1 and replay_pi==1:
#                     print("444")
                    with torch.no_grad():
                        real=torch.Tensor(np.array([replay_reward]))+model_1(y_train)
                    pred=model_1(x_train)
                    loss=loss_fn(pred,real)
        
                    
                    optimizer_1.zero_grad()
                    loss.backward()
                    optimizer_1.step()
                    res=max(res,loss.item())
            with torch.no_grad():
                if replay_pi_old==1:
                    for param0, param1 in zip(model_0.parameters(), model_1.parameters()):
                        for k in range(1,1504):
                            if k==1 or k==752:
                                if param1[0][k] > param0[0][k]:
                                    param1[0][k]=param0[0][k].clone()
                elif replay_pi_old==0:
                    for param0, param1 in zip(model_0.parameters(), model_1.parameters()):
                        for k in range(1,1504):
                            if k==1 or k==752:
                                if param1[0][k] > param0[0][k]:
                                    param0[0][k]=param1[0][k].clone()      
    
    print("Total_value:",total_value)
    print("剩余预算:",B_i)
    print("TD-error",res)
    print(" ")
    loss_abc.append(math.sqrt(res))
print(nt)


epoch: 0
Total_value: 1692.8942508818322
剩余预算: 4772.6438019229345
TD-error 0
 
epoch: 1
Total_value: 14289.923866463454
剩余预算: 3080.859002623764
TD-error 1143358.25
 
epoch: 2
Total_value: 12670.252800025874
剩余预算: 3298.3813053953972
TD-error 1370848.375
 
epoch: 3
Total_value: 13458.371691628223
剩余预算: 3192.536705395082
TD-error 891373.4375
 
epoch: 4
Total_value: 13646.490128160494
剩余预算: 3167.272343787202
TD-error 1326738.375
 
epoch: 5
Total_value: 11911.252272527685
剩余预算: 3400.3153004931955
TD-error 847173.375
 
epoch: 6
Total_value: 13897.973004116388
剩余预算: 3133.4981192430314
TD-error 1656288.875
 
epoch: 7
Total_value: 12476.130022621728
剩余预算: 3324.452051756414
TD-error 1980410.5
 
epoch: 8
Total_value: 13583.570600907284
剩余预算: 3175.722454887552
TD-error 776379.5625
 
epoch: 9
Total_value: 14118.452681718412
剩余预算: 3103.887633398011
TD-error 842426.875
 
epoch: 10
Total_value: 12909.113494343783
剩余预算: 3266.302243574567
TD-error 1052673.0
 
epoch: 11
Total_value: 13558.128913129513
剩余

Total_value: 13132.015039230953
剩余预算: 3236.366500237584
TD-error 3974861.75
 
epoch: 94
Total_value: 13658.94148218993
剩余预算: 3165.600123262164
TD-error 4273716.5
 
epoch: 95
Total_value: 13491.654577367304
剩余预算: 3188.066804006541
TD-error 7943766.0
 
epoch: 96
Total_value: 12287.263085706074
剩余预算: 3349.816937186937
TD-error 5461138.0
 
epoch: 97
Total_value: 11732.172460645525
剩余预算: 3424.3657721400073
TD-error 820185.3125
 
epoch: 98
Total_value: 13433.71355897539
剩余预算: 3195.8482998958657
TD-error 6186516.5
 
epoch: 99
Total_value: 13250.386495076118
剩余预算: 3220.4691787434713
TD-error 5126641.0
 
epoch: 100
Total_value: 12302.829956029467
剩余预算: 3347.726301903106
TD-error 6582660.5
 
epoch: 101
Total_value: 13295.373344774795
剩余预算: 3214.427431537095
TD-error 8468819.0
 
epoch: 102
Total_value: 13425.593281335496
剩余预算: 3196.9388555821242
TD-error 10580611.0
 
epoch: 103
Total_value: 12523.477137565118
剩余预算: 3318.0933202303086
TD-error 4401315.5
 
epoch: 104
Total_value: 14187.647152323016

Total_value: 13636.783509518438
剩余预算: 3168.575945538754
TD-error 13750096.0
 
epoch: 187
Total_value: 12588.217880349068
剩余预算: 3309.3986193460833
TD-error 9924844.0
 
epoch: 188
Total_value: 13114.42150656207
剩余预算: 3238.729316873211
TD-error 6750828.0
 
epoch: 189
Total_value: 13703.561074727635
剩余预算: 3159.6076988010154
TD-error 8635161.0
 
epoch: 190
Total_value: 11738.637426635345
剩余预算: 3423.49752529743
TD-error 10396867.0
 
epoch: 191
Total_value: 13558.633922835119
剩余预算: 3179.0714581204525
TD-error 15320864.0
 
epoch: 192
Total_value: 13397.137675831114
剩余预算: 3200.7604518088765
TD-error 10675898.0
 
epoch: 193
Total_value: 12881.865775561351
剩余预算: 3269.9616202576753
TD-error 12307144.0
 
epoch: 194
Total_value: 13469.227718297336
剩余预算: 3191.0787378058903
TD-error 9705037.0
 
epoch: 195
Total_value: 13966.192822117842
剩余预算: 3124.336177529164
TD-error 9646998.0
 
epoch: 196
Total_value: 13815.582817982813
剩余预算: 3144.563145583829
TD-error 7727280.0
 
epoch: 197
Total_value: 14192.5805

Total_value: 14173.380827694953
剩余预算: 3096.510767164256
TD-error 15828619.0
 
epoch: 279
Total_value: 13416.382499605439
剩余预算: 3198.1758662898947
TD-error 32881484.0
 
epoch: 280
Total_value: 14037.730426575485
剩余预算: 3114.728656113956
TD-error 21163338.0
 
epoch: 281
Total_value: 13141.426569848365
剩余预算: 3235.1025288949277
TD-error 25610668.0
 
epoch: 282
Total_value: 13884.88594959561
剩余预算: 3135.2557145318838
TD-error 25274794.0
 
epoch: 283
Total_value: 13687.487589840923
剩余预算: 3161.7663725703833
TD-error 23381006.0
 
epoch: 284
Total_value: 13434.81906273897
剩余预算: 3195.699830413785
TD-error 19225614.0
 
epoch: 285
Total_value: 12310.464168240947
剩余预算: 3346.7010249474947
TD-error 612363.5
 
epoch: 286
Total_value: 13734.576793664497
剩余预算: 3155.442278583872
TD-error 15700348.0
 
epoch: 287
Total_value: 13496.15898700209
剩余预算: 3187.4618604617126
TD-error 6517829.5
 
epoch: 288
Total_value: 13791.786871319373
剩余预算: 3147.758948251494
TD-error 30211764.0
 
epoch: 289
Total_value: 13235.33

Total_value: 13483.898524750235
剩余预算: 3189.1084441646194
TD-error 22246730.0
 
epoch: 371
Total_value: 13791.786871319373
剩余预算: 3147.758948251494
TD-error 15366257.0
 
epoch: 372
Total_value: 14082.896056907066
剩余预算: 3108.662898615758
TD-error 25291414.0
 
epoch: 373
Total_value: 12287.263085706074
剩余预算: 3349.816937186937
TD-error 11170314.0
 
epoch: 374
Total_value: 13526.751322913715
剩余预算: 3183.3533007099504
TD-error 9278151.0
 
epoch: 375
Total_value: 14201.010786316867
剩余预算: 3092.800055557766
TD-error 8158019.0
 
epoch: 376
Total_value: 13165.560581552876
剩余预算: 3231.8613239923598
TD-error 15825724.0
 
epoch: 377
Total_value: 13120.185090137162
剩余预算: 3237.9552658961657
TD-error 14793342.0
 
epoch: 378
Total_value: 13026.586991484688
剩余预算: 3250.525518199745
TD-error 13343569.0
 
epoch: 379
Total_value: 13874.350534784231
剩余预算: 3136.6706238538513
TD-error 29879228.0
 
epoch: 380
Total_value: 13588.117116721354
剩余预算: 3175.111856470407
TD-error 14760697.0
 
epoch: 381
Total_value: 14342

...

Total_value: 11172.530491256519
剩余预算: 3499.5258539811275
TD-error 94527.5546875
 
epoch: 4637
Total_value: 13260.703064993475
剩余预算: 3219.083660355429
TD-error 311041.71875
 
epoch: 4638
Total_value: 11716.967509628043
剩余预算: 3426.4078015541186
TD-error 339439.71875
 
epoch: 4639
Total_value: 13473.601478432023
剩余预算: 3190.4913405275283
TD-error 260153.609375
 
epoch: 4640
Total_value: 13453.159255849501
剩余预算: 3193.236737060234
TD-error 44213.04296875
 
epoch: 4641
Total_value: 12992.964641533817
剩余预算: 3255.04100973223
TD-error 240557.8125
 
epoch: 4642
Total_value: 12369.459248858093
剩余预算: 3338.777968189889
TD-error 32755.001953125
 
epoch: 4643
Total_value: 13065.408436501733
剩余预算: 3245.3117866637476
TD-error 78091.2109375
 
epoch: 4644
Total_value: 11911.252272527685
剩余预算: 3400.3153004931955
TD-error 71768.6953125
 
epoch: 4645
Total_value: 12742.4223374281
剩余预算: 3288.6889151990204
TD-error 64217.71875
 
epoch: 4646
Total_value: 13252.120187017605
剩余预算: 3220.2363434034896
TD-error 1088

Total_value: 13090.188739926665
剩余预算: 3241.983784592173
TD-error 64739.546875
 
epoch: 4726
Total_value: 13460.666017822807
剩余预算: 3192.228576709267
TD-error 274746.0625
 
epoch: 4727
Total_value: 13070.539675787319
剩余预算: 3244.622659711612
TD-error 68957.515625
 
epoch: 4728
Total_value: 12797.257943972623
剩余预算: 3281.324477038326
TD-error 145229.96875
 
epoch: 4729
Total_value: 12504.179655006354
剩余预算: 3320.684977839599
TD-error 48398.7734375
 
epoch: 4730
Total_value: 12633.028276308189
剩余预算: 3303.3805699290647
TD-error 178967.890625
 
epoch: 4731
Total_value: 12708.154319669482
剩余预算: 3293.2911201088546
TD-error 28660.013671875
 
epoch: 4732
Total_value: 13404.581166956674
剩余预算: 3199.7607887514537
TD-error 120685.3515625
 
epoch: 4733
Total_value: 13290.692252119765
剩余预算: 3215.0561036637446
TD-error 92005.421875
 
epoch: 4734
Total_value: 9666.632419457432
剩余预算: 3701.768409957368
TD-error 98941.2421875
 
epoch: 4735
Total_value: 13381.15331619912
剩余预算: 3202.9071560302027
TD-error 95619

Total_value: 12999.892499126021
剩余预算: 3254.110596410687
TD-error 25083.1875
 
epoch: 4815
Total_value: 13513.020396530785
剩余预算: 3185.197368180124
TD-error 117638.078125
 
epoch: 4816
Total_value: 13579.378177131663
剩余预算: 3176.2854986393136
TD-error 21081.0234375
 
epoch: 4817
Total_value: 13176.965554187957
剩余预算: 3230.3296327977505
TD-error 89874.75
 
epoch: 4818
Total_value: 12483.115486760174
剩余预算: 3323.5139018586933
TD-error 64111.37109375
 
epoch: 4819
Total_value: 11553.826582585887
剩余预算: 3448.317676257604
TD-error 75834.5859375
 
epoch: 4820
Total_value: 12592.694928931916
剩余预算: 3308.797350398615
TD-error 55224.88671875
 
epoch: 4821
Total_value: 11123.363729457873
剩余预算: 3506.1289646175296
TD-error 101323.84375
 
epoch: 4822
Total_value: 13129.150972877587
剩余预算: 3236.7511451950604
TD-error 142003.90625
 
epoch: 4823
Total_value: 13507.837882871794
剩余预算: 3185.8933812957575
TD-error 23438.240234375
 
epoch: 4824
Total_value: 12689.096293041326
剩余预算: 3295.85061871591
TD-error 148426

Total_value: 13700.957001529934
剩余预算: 3159.957426600869
TD-error 150245.125
 
epoch: 4904
Total_value: 13730.73741095906
剩余预算: 3155.9579088155992
TD-error 74828.578125
 
epoch: 4905
Total_value: 13733.167590695159
剩余预算: 3155.631534959018
TD-error 24464.978515625
 
epoch: 4906
Total_value: 11998.172845249255
剩余预算: 3388.641841895078
TD-error 44690.57421875
 
epoch: 4907
Total_value: 11794.380610558324
剩余预算: 3416.011199226657
TD-error 24463.68359375
 
epoch: 4908
Total_value: 13396.490569261015
剩余预算: 3200.847358412434
TD-error 21313.865234375
 
epoch: 4909
Total_value: 12979.495450994022
剩余预算: 3256.849926001339
TD-error 45084.4140625
 
epoch: 4910
Total_value: 13374.393166922608
剩余预算: 3203.815046075394
TD-error 21388.533203125
 
epoch: 4911
Total_value: 13006.498711918224
剩余预算: 3253.2233800808194
TD-error 155259.421875
 
epoch: 4912
Total_value: 12984.205687051985
剩余预算: 3256.217339907066
TD-error 152480.328125
 
epoch: 4913
Total_value: 13093.999213823618
剩余预算: 3241.47203682197
TD-error 8

Total_value: 12514.245324142628
剩余预算: 3319.3331555005866
TD-error 88925.8203125
 
epoch: 4993
Total_value: 13591.52483287855
剩余预算: 3174.6541991836502
TD-error 33583.51171875
 
epoch: 4994
Total_value: 13290.58728016454
剩余预算: 3215.070201428344
TD-error 76997.4609375
 
epoch: 4995
Total_value: 12437.119415824405
剩余预算: 3329.6911877753937
TD-error 34500.30859375
 
epoch: 4996
Total_value: 13192.756693047657
剩余预算: 3228.208878183232
TD-error 24083.265625
 
epoch: 4997
Total_value: 13041.118623400234
剩余预算: 3248.573915739964
TD-error 46238.87890625
 
epoch: 4998
Total_value: 13589.299567707752
剩余预算: 3174.953052953564
TD-error 108947.8359375
 
epoch: 4999
Total_value: 13838.618745016443
剩余预算: 3141.4694137770025
TD-error 227755.8125
 
[6793, 6863, 100200, 22083]


In [1]:
for i in range(2000):
    print('epoch:',i)
    idx=random.randrange(0,100)
    roadpath=str(idx)+'_road.xlsx'
    roadpath=os.path.join(rootpath,roadpath)
    road=pd.read_excel(roadpath)
    road=road.to_numpy()
    
    bpath=str(idx)+'_b.xlsx'
    bpath=os.path.join(rootpath,bpath)
    b=pd.read_excel(bpath,header=None)
    b=b.to_numpy().flatten()
    
    path=r'C:\Users\98306\Desktop\实验\Real map\poi次数分布(总次数为1241).xlsx'
    df=pd.read_excel(path)
    arr_df = df.to_numpy().flatten()

    B_i=B
    res=0#记录每轮最大的TD-error
    total_value=0#每轮获得的总价值
    
    for j in range(road.shape[0]):
        for k in range(road.shape[1]):
            if arr_df[k]==0:
                road[j,k]=0
        if road[j].sum()*danwei_b<b[j]:
            continue
        pi=0
        reward=0
        s= np.array([]).astype(np.double)
        s= np.append(s,1)
        s= np.append(s,j)
        s= np.concatenate((s, road[j]))
        s= np.append(s,b[j])
        s= np.concatenate((s,arr_df))
        s= np.append(s,B_i)
        epsilon=0.1
        lamda=random.uniform(0, 1)
        if lamda<epsilon/2:#随机不选
            pi=0
            reward=0
            nt[0]+=1
        elif lamda<epsilon:#随机选择
            pi=1
            reward=road[j].sum()
            nt[1]+=1
        else:
            with torch.no_grad():
                t0=model_0(torch.Tensor(s))
                t1=model_1(torch.Tensor(s))
                t0=t0.detach().numpy()
                t1=t1.detach().numpy()
                
                if t0[0]<t1[0]:
                    pi=1
                    reward=road[j].sum()
                    nt[2]+=1
                else:
                    pi=0
                    reward=0
                    nt[3]+=1

        if j>=1:#存入经验回放数组
            exper=[]
            exper.append(s_old)
            exper.append(pi_old)
            exper.append(reward_old)
            exper.append(s)
            exper.append(pi)
            experience.append(exper)
#         if j==road.shape[0]-1:
#             exper=[]
#             exper.append(s)
#             exper.append(pi)
#             exper.append(reward)
#             exper.append(np.array([0]))
#             exper.append(0)
#             experience.append(exper)
            
        s_old=s
        pi_old=pi
        reward_old=reward
        
        if pi==1:
            for k in range(road.shape[1]):
                if road[j,k]>0:
                    arr_df[k]-=1
            B_i-=road[j].sum()*danwei_b
            

        
        total_value+=reward
        if len(experience)>=1000:
            experience.pop(0)
        if len(experience)>100:
            idx=random.randrange(0,len(experience))#随机抽取一条经验
            replay_s_old=experience[idx][0]#S
            replay_pi_old=experience[idx][1]#A
            replay_reward=experience[idx][2]#R
            replay_s=experience[idx][3]#S
            replay_pi=experience[idx][4]#A
#             if replay_s.shape[0]==1:
#                 x_train=torch.Tensor(replay_s_old)
#                 real=torch.Tensor(np.array([replay_reward]))
#                 if replay_pi_old==0:
#                     pred=model_0(x_train)
#                     loss=loss_fn(pred,real)
                    
#                     optimizer_0.zero_grad()
#                     loss.backward()
#                     optimizer_0.step()
#                     res=max(res,loss.item())
#                 else:
#                     pred=model_1(x_train)
#                     loss=loss_fn(pred,real)
                    
#                     optimizer_1.zero_grad()
#                     loss.backward()
#                     optimizer_1.step()
#                     res=max(res,loss.item())
#             else:
            if replay_s.shape[0]!=1:
                x_train=torch.Tensor(replay_s_old)
                y_train=torch.Tensor(replay_s)
                if replay_pi_old==0 and replay_pi==0:
#                     print("111")
                    with torch.no_grad():
                        real=torch.Tensor(np.array([replay_reward]))+model_0(y_train)
                    pred=model_0(x_train)
                    loss=loss_fn(pred,real)
                    
                    optimizer_0.zero_grad()
                    loss.backward()
                    optimizer_0.step()
                    res=max(res,loss.item())
                elif replay_pi_old==0 and replay_pi==1:
#                     print("222")
                    with torch.no_grad():
                        real=torch.Tensor(np.array([replay_reward]))+model_1(y_train)
                    pred=model_0(x_train)
                    loss=loss_fn(pred,real)
                    
                    optimizer_0.zero_grad()
                    loss.backward()
                    optimizer_0.step()
                    res=max(res,loss.item())
                elif replay_pi_old==1 and replay_pi==0:
#                     print("333")
                    with torch.no_grad():
                        real=torch.Tensor(np.array([replay_reward]))+model_0(y_train)
#                         print(real,torch.Tensor(np.array([replay_reward])),model_0(y_train))
                    pred=model_1(x_train)
                    loss=loss_fn(pred,real)
                    
                    optimizer_1.zero_grad()
                    loss.backward()
                    optimizer_1.step()
                    res=max(res,loss.item())
                elif replay_pi_old==1 and replay_pi==1:
#                     print("444")
                    with torch.no_grad():
                        real=torch.Tensor(np.array([replay_reward]))+model_1(y_train)
                    pred=model_1(x_train)
                    loss=loss_fn(pred,real)
        
                    
                    optimizer_1.zero_grad()
                    loss.backward()
                    optimizer_1.step()
                    res=max(res,loss.item())
            with torch.no_grad():
                if replay_pi_old==1:
                    for param0, param1 in zip(model_0.parameters(), model_1.parameters()):
                        for k in range(1,1504):
                            if k==1 or k==752:
                                if param1[0][k] > param0[0][k]:
                                    param1[0][k]=param0[0][k].clone()
                elif replay_pi_old==0:
                    for param0, param1 in zip(model_0.parameters(), model_1.parameters()):
                        for k in range(1,1504):
                            if k==1 or k==752:
                                if param1[0][k] > param0[0][k]:
                                    param0[0][k]=param1[0][k].clone()      
    
    print("Total_value:",total_value)
    print("剩余预算:",B_i)
    print("TD-error",res)
    print(" ")
    loss_abc.append(math.sqrt(res))
print(nt)

epoch: 0


NameError: name 'random' is not defined